In [ ]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [ ]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [3]:
pmid = plib.doi2pmid("10.1093/cercor/bhn229")
print(pmid)

KeyboardInterrupt: 